# Tarea 03 - Para entregar

Se trata de aplicar los cuatro métodos de ajustar y validar el modelo, razonando detalladamente en el informe la justificación de elegir un modelo:

1. Clasificación en un problema con dos clases.

2. Clasificación en un problema con más de dos clases.

2. Regresión: seleccionar por crossvalidación un conjunto óptimo de predictores usando, por ejemplo, la estrategia "forward" (para menor engorro) comparando con el resultado de `step`.



CV variancia alta, Bootstrap bias alto, LOO alto coste computacional

Muchos datos = la variancia no es un problema = CV es mejor/su defecto pierde peso

In [1]:
#install.packages("class",dependencies=TRUE,repos="https://cloud.r-project.org")
require(class)

Loading required package: class


In [2]:
kfoldIndexes<-function(n,k){
    l<-floor(n/k)
    Indexes<-c(1,(1:k)*l)
    Indexes[k+1]<-n
    return(Indexes)
    }


# lista de las "k" del knn
ks<-c(3,5,7)

### 1. Clasificador de dos clases

In [3]:
require(ElemStatLearn)
data(spam)
spam$spam <- as.factor(spam$spam)

set<-spam

Loading required package: ElemStatLearn
Warning message:
"package 'ElemStatLearn' was built under R version 3.4.4"

In [4]:
n<-nrow(set)

# Fijamos un 'K' para k-fold cross-validation (pongo mayúscula para no confundir con la k de knn)
K<-5
J<-kfoldIndexes(n,K)
J
Lower<-J[-(K+1)]
Upper<-J[-1]
Lower
Upper
# Una permutación aleatoria de los índices
I<-sample(1:n)

[1]    1  920 1840 2760 3680 4601

[1]    1  920 1840 2760 3680

[1]  920 1840 2760 3680 4601

In [5]:
#cross ajustado a las columnas del dataset spam

for (k in ks){
    
    P.ERR<-rep(0,K)
    for (fold in 1:K){
        Itest<-I[Lower[fold]:(Upper[fold]-1)]
        set.test<-set[Itest,]
        set.train<-set[-Itest,]
        Xtrain<-as.matrix(set.train[,-length(names(set))])
        ytrain<-set.train[,length(names(set))]
        Xtest<-as.matrix(set.test[,-length(names(set))])
        ytest<-set.test[,length(names(set))]
        y.hat<-knn(Xtrain,Xtest,ytrain,k)
        C<-table("True"=ytest,"Predicted"=y.hat)
        ntest<-sum(C)
        P.ERR[fold]<-(ntest-sum(diag(C)))/ntest
        }

    print(paste("K= ", k))
    print(round(P.ERR,3))
    mean.p.err<-mean(P.ERR)
    print(round(mean.p.err,3))
}

[1] "K=  3"
[1] 0.195 0.190 0.207 0.184 0.185
[1] 0.192
[1] "K=  5"
[1] 0.198 0.203 0.204 0.192 0.198
[1] 0.199
[1] "K=  7"
[1] 0.200 0.214 0.214 0.193 0.205
[1] 0.205


In [6]:
#LOO ajustado a las columnas del dataset spam

for (k in ks){
    
    g<-nlevels(set$spam)
    C<-matrix(0,nrow=g,ncol=g)
    for (i in 1:n){
        set.test<-set[i,]
        set.train<-set[-i,]
        Xtrain<-as.matrix(set.train[,-length(names(set))])
        ytrain<-set.train[,length(names(set))]
        Xtest<-as.matrix(set.test[,-length(names(set))])
        ytest<-set.test[,length(names(set))]
        y.hat<-knn(Xtrain,Xtest,ytrain,k)
        C[ytest,y.hat]=C[ytest,y.hat]+1
        }
    
    print(paste("K= ", k))
    p.err<-(n-sum(diag(C)))/n
    print(round(p.err,3))
    
}

[1] "K=  3"
[1] 0.187
[1] "K=  5"
[1] 0.188
[1] "K=  7"
[1] 0.197


In [7]:
#Boot ajustado a las columnas del dataset spam

for (k in ks){
    
    I<-1:n
    # Número de remuestras bootstrap
    B<-10
    P.ERR<-rep(0,B)
    for (b in 1:B){
        Ib<-sample(I,n,replace = TRUE)
        oob<-I[is.na(match(I,Ib))]
        Itest<-oob
        ntest<-length(oob)
        set.test<-set[Itest,]
        set.train<-set[-Itest,]
        Xtrain<-as.matrix(set.train[,-length(names(set))])
        ytrain<-set.train[,length(names(set))]
        Xtest<-as.matrix(set.test[,-length(names(set))])
        ytest<-set.test[,length(names(set))]
        y.hat<-knn(Xtrain,Xtest,ytrain,k)
        C<-table("True"=ytest,"Predicted"=y.hat)
        P.ERR[b]<-(ntest-sum(diag(C)))/ntest
        }
    
    print(paste("K= ", k))
    print(round(P.ERR,3))
    mean.p.err<-mean(P.ERR)
    print(round(mean.p.err,3))
    
}

[1] "K=  3"
 [1] 0.198 0.208 0.202 0.198 0.216 0.211 0.210 0.199 0.218 0.209
[1] 0.207
[1] "K=  5"
 [1] 0.204 0.206 0.201 0.215 0.216 0.199 0.198 0.204 0.205 0.218
[1] 0.207
[1] "K=  7"
 [1] 0.206 0.204 0.222 0.213 0.205 0.217 0.202 0.209 0.235 0.212
[1] 0.213


Los valores de k (3, 5, 7) son elegidos a dedo basados en lo observado en la teoria, no son valores muy altos ni 1 o 2.

El mejor parece LOO pero tarda una eternidad (es lo normal, es un metodo muy pesado con muchas iteraciones y modelos).

En mi opinion es mejor el CV, no es lento ni el peor. En cuanto a la "k", no hay mucha diferencia entre usar 3 o 5 aunque puede que sea caer en el overfitting. 7 da unos resultados algo peores, pero mas o menos despreciable en este caso (muy pequeña desviacion en el CV), y a la larga ese clasificador no caera tanto en el overfitting.

### 2. Clasificador de mas de dos clases

In [8]:
require(ElemStatLearn)
data(iris)
str(iris)

set<-iris

'data.frame':	150 obs. of  5 variables:
 $ Sepal.Length: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ Sepal.Width : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ Petal.Length: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ Petal.Width : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ Species     : Factor w/ 3 levels "setosa","versicolor",..: 1 1 1 1 1 1 1 1 1 1 ...


In [9]:
n<-nrow(set)

# Fijamos un 'K' para k-fold cross-validation (pongo mayúscula para no confundir con la k de knn)
K<-5
J<-kfoldIndexes(n,K)
J
Lower<-J[-(K+1)]
Upper<-J[-1]
Lower
Upper
# Una permutación aleatoria de los índices
I<-sample(1:n)

[1]   1  30  60  90 120 150

[1]   1  30  60  90 120

[1]  30  60  90 120 150

In [10]:
#cross
for (k in ks){
    P.ERR<-rep(0,K)
    for (fold in 1:K){
        Itest<-I[Lower[fold]:(Upper[fold]-1)]
        set.test<-set[Itest,]
        set.train<-set[-Itest,]
        Xtrain<-as.matrix(set.train[,-length(names(set))])
        ytrain<-set.train[,length(names(set))]
        Xtest<-as.matrix(set.test[,-length(names(set))])
        ytest<-set.test[,length(names(set))]
        y.hat<-knn(Xtrain,Xtest,ytrain,k)
        C<-table("True"=ytest,"Predicted"=y.hat)
        ntest<-sum(C)
        P.ERR[fold]<-(ntest-sum(diag(C)))/ntest
    }
    print(paste("K= ", k))
    print(round(P.ERR,3))
    mean.p.err<-mean(P.ERR)
    print(round(mean.p.err,3))
}

[1] "K=  3"
[1] 0.000 0.100 0.033 0.000 0.033
[1] 0.033
[1] "K=  5"
[1] 0.000 0.033 0.100 0.000 0.033
[1] 0.033
[1] "K=  7"
[1] 0.000 0.033 0.100 0.000 0.067
[1] 0.04


In [11]:
#LOO
for (k in ks){
    g<-nlevels(set$Species)
    C<-matrix(0,nrow=g,ncol=g)
    for (i in 1:n){
        set.test<-set[i,]
        set.train<-set[-i,]
        Xtrain<-as.matrix(set.train[,-length(names(set))])
        ytrain<-set.train[,length(names(set))]
        Xtest<-as.matrix(set.test[,-length(names(set))])
        ytest<-set.test[,length(names(set))]
        y.hat<-knn(Xtrain,Xtest,ytrain,k)
        C[ytest,y.hat]=C[ytest,y.hat]+1
    }
    print(paste("K= ", k))
    p.err<-(n-sum(diag(C)))/n
    print(round(p.err,3))
}

[1] "K=  3"
[1] 0.04
[1] "K=  5"
[1] 0.033
[1] "K=  7"
[1] 0.027


In [12]:
#Boot
for (k in ks){
    I<-1:n
    # Número de remuestras bootstrap
    B<-10
    P.ERR<-rep(0,B)
    for (b in 1:B){
        Ib<-sample(I,n,replace = TRUE)
        oob<-I[is.na(match(I,Ib))]
        Itest<-oob
        ntest<-length(oob)
        set.test<-set[Itest,]
        set.train<-set[-Itest,]
        Xtrain<-as.matrix(set.train[,-length(names(set))])
        ytrain<-set.train[,length(names(set))]
        Xtest<-as.matrix(set.test[,-length(names(set))])
        ytest<-set.test[,length(names(set))]
        y.hat<-knn(Xtrain,Xtest,ytrain,k)
        C<-table("True"=ytest,"Predicted"=y.hat)
        P.ERR[b]<-(ntest-sum(diag(C)))/ntest
    }
    print(paste("K= ", k))
    print(round(P.ERR,3))
    mean.p.err<-mean(P.ERR)
    print(round(mean.p.err,3))
}

[1] "K=  3"
 [1] 0.034 0.038 0.077 0.055 0.036 0.017 0.056 0.052 0.016 0.017
[1] 0.04
[1] "K=  5"
 [1] 0.036 0.056 0.036 0.057 0.018 0.000 0.038 0.035 0.018 0.064
[1] 0.036
[1] "K=  7"
 [1] 0.017 0.000 0.035 0.017 0.038 0.054 0.075 0.035 0.034 0.021
[1] 0.033


Para este caso el mejor valor de "k" para el knn en los tres casos es el 5, puesto que 3 y 7 dan mas error en los tres metodos como minimo sabemos que llegamos a un minimo local para el error, pero es bueno porque es mejor que 3 que nos plantearia mas claro el problema de overfitting.

Los tres metodos dan unos resultados muy buenos, aunque esto puede deberse a overfitting debido a que hay pocas muestras en el dataset y/o a la poca varianza en los datos.

El mejor otra vez es LOO, en este caso tiene mas sentido usarlo debido a la escasa cantidad de muestras. Por el mismo motivo, bootstrap da mejor resultado que el CV pero si hay que escoger uno para este ejemplo escogeriamos el LOO por el bajo error que ha dado y por tener un volumen de datos reducido.

### 3. Regresion seleccionando por cross-validation un conjunto optimo de predictores

In [13]:
require(ISLR)
data(Smarket)
str(Smarket)

set<-Smarket

Loading required package: ISLR
Warning message:
"package 'ISLR' was built under R version 3.4.4"

'data.frame':	1250 obs. of  9 variables:
 $ Year     : num  2001 2001 2001 2001 2001 ...
 $ Lag1     : num  0.381 0.959 1.032 -0.623 0.614 ...
 $ Lag2     : num  -0.192 0.381 0.959 1.032 -0.623 ...
 $ Lag3     : num  -2.624 -0.192 0.381 0.959 1.032 ...
 $ Lag4     : num  -1.055 -2.624 -0.192 0.381 0.959 ...
 $ Lag5     : num  5.01 -1.055 -2.624 -0.192 0.381 ...
 $ Volume   : num  1.19 1.3 1.41 1.28 1.21 ...
 $ Today    : num  0.959 1.032 -0.623 0.614 0.213 ...
 $ Direction: Factor w/ 2 levels "Down","Up": 2 2 1 2 2 2 1 2 2 2 ...


In [14]:
set0<-glm(Direction~1,data=set,family=binomial)
set1<-glm(Direction~.,data=set,family=binomial)

step.result<-step(set0,scope=formula(set1), direction="forward", trace=0)

Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"Warning message:
"glm.fit: algorithm did not converge"Warning message:
"glm.fit: fitted pr

In [15]:
step.result


Call:  glm(formula = Direction ~ Today, family = binomial, data = set)

Coefficients:
(Intercept)        Today  
      8.421    17192.414  

Degrees of Freedom: 1249 Total (i.e. Null);  1248 Residual
Null Deviance:	    1731 
Residual Deviance: 0.0007505 	AIC: 4.001

In [16]:
set.pred<-predict(step.result,newdata=set,type="response")
str(set.pred)

 Named num [1:1250] 1.00 1.00 2.22e-16 1.00 1.00 ...
 - attr(*, "names")= chr [1:1250] "1" "2" "3" "4" ...


In [17]:
set.pred.crisp<-1*(set.pred>=0.5)
C<-table("True"=set$Direction,"Predicted"=set.pred.crisp)
C

      Predicted
True     0   1
  Down 602   0
  Up     0 648

Lo realizado mas arriba con la funcion step() es una feature selection hacia  adelante ("forward").

Haria una funcion para hacer feature selection explicitamente con k-fold cross-validation. La funcion haria lo siguiente:

- Empezariamos con ninguna variable en el modelo

- Usamos el metodo de cross-validation usado en los apartados anteriores, con cada variable del modelo original y cogemos la que tiene la media de error mas baja. Añadimos esa variable al modelo.

- Usamos el nuevo modelo + las demas variables; hasta que el error no decrementa o, el decrecimiento es demasiado bajo para que merezca la pena añadirlo al modelo.

In [18]:
kfoldCV<-function(set,kn,Kf){
    n<-nrow(set)
    # Fijamos un 'K' para k-fold cross-validation (pongo mayúscula para no confundir con la k de knn)
    k<-kn
    K<-Kf
    J<-kfoldIndexes(n,K)
    J
    Lower<-J[-(K+1)]
    Upper<-J[-1]
    Lower
    Upper
    # Una permutación aleatoria de los índices
    I<-sample(1:n)

    #cross
        P.ERR<-rep(0,K)
        for (fold in 1:K){
            Itest<-I[Lower[fold]:(Upper[fold]-1)]
            set.test<-set[Itest,]
            set.train<-set[-Itest,]
            Xtrain<-as.matrix(set.train[,-length(names(set))])
            ytrain<-set.train[,length(names(set))]
            Xtest<-as.matrix(set.test[,-length(names(set))])
            ytest<-set.test[,length(names(set))]
            y.hat<-knn(Xtrain,Xtest,ytrain,k)
            C<-table("True"=ytest,"Predicted"=y.hat)
            ntest<-sum(C)
            P.ERR[fold]<-(ntest-sum(diag(C)))/ntest
        }
        mean.p.err<-mean(P.ERR)
        return(round(mean.p.err,3))
    }

In [19]:
set<-Smarket

features<-c(1:(length(names(set))-1))
y<-length(names(set))
print(features)
print(y)

kn<-7
Kf<-5
featureList<-c(y)

[1] 1 2 3 4 5 6 7 8
[1] 9


In [20]:
errors<-c()
for(i in features){
    columns<-c()
    columns<-c(i,featureList)
    modelo<-set[columns]
    errors<-c(errors, kfoldCV(modelo,kn,Kf))
}
print(min(errors))
indexFound<-match(min(errors),errors)
featureList<-c(features[indexFound], featureList)
features<-features[-indexFound]
print(features)
print(featureList)
print(names(set[featureList]))

[1] 0.001
[1] 1 2 3 4 5 6 7
[1] 8 9
[1] "Today"     "Direction"


No soy muy diestro en R, creo que por eso no se continuar hasta la convergencia de forma automatica, pero el metodo consistiria en seguir ejecutando el programa del recuadro de arriba hasta decidir si estamos contentos con el error que provoca.